In [ ]:
import time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/Colab/'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

#Read Data

In [ ]:
train_labels = pd.read_csv('../data/1-original-data/train_labels.csv')
print(train_labels.shape)
train_labels

FileNotFoundError: ignored

In [ ]:
train_df = pd.read_parquet('../data/1-original-data/train.parquet', engine = 'pyarrow')
print(train_df.shape)
len(train_df.columns)

In [ ]:
tmp = train_df.merge(train_labels,on='customer_ID',how='left')

In [ ]:
y_full = tmp.iloc[:,-1]
print(y_full.shape)

Drop columns with too many missing values/categorical variables that are not good predictors of defaulting

In [ ]:
train_df.drop(columns = ["B_31", "D_87", "D_88", "D_110", "B_39", "D_73", "B_42", "D_134", "B_29", "D_132", "D_76", "D_42", "D_142", "D_116", "D_63", "D_126", "D_66"], inplace = True)
len(train_df.columns)

Fill missing data (with RandomForest)

In [ ]:
train_df_randomForest = train_df.copy()

In [ ]:
tmp = train_df_randomForest.loc[:,['customer_ID','S_2']]
tmp.head(1)

In [ ]:
train_df_randomForest.drop(columns = ['customer_ID','S_2'], inplace = True)

In [ ]:
train_df_randomForest.columns

In [ ]:
np.argsort(train_df_randomForest.isnull().sum(axis=0))

In [ ]:
sort_index = np.argsort(train_df_randomForest.isnull().sum(axis=0)).values

In [ ]:
for i in sort_index:
    t = time.time() # 记录时间
    
    # 构建新特征矩阵和新标签
    df = train_df_randomForest
    fillc = df.iloc[:,i] #即将被填充的特征，视为新目标变量
    ytrain = fillc[fillc.notnull()]
    ytest = fillc[fillc.isnull()]
    if len(ytest)== 0:
        continue
    df = pd.concat([df.iloc[:, :i], df.iloc[:, i+1:], y_full], axis=1) #把新目标变量剔除，然后把原始标签进新特征矩阵
    # 在新特征矩阵中，对含有缺失值的列，用0进行填补。得到最终版特征矩阵
    df_0 = df.fillna(0)
    # # 找出能够用来训练的资料 以及 将要进行填补的资料
    Xtrain = df_0.iloc[ytrain.index, :]
    Xtest = df_0.iloc[ytest.index, :]
    # 随机森林训练以及预测
    # rfr = RandomForestRegressor(n_estimators=50, max_depth = 10, max_features = 50)
    rfr = DecisionTreeRegressor(max_depth = 10)
    rfr = rfr.fit(Xtrain, ytrain)
    ypredict = rfr.predict(Xtest)
    # # 将填补结果放回数据集
    train_df_randomForest.iloc[train_df_randomForest.iloc[:,i].isnull(),i] = ypredict

    # 追踪记录
    dur = round((time.time() - t) / 60, 2)
    print(f"当前已完成第{i}个列的缺失值填补， 消耗时间 {dur} mins")

当前已完成第73个列的缺失值填补， 消耗时间 5.76 mins
当前已完成第93个列的缺失值填补， 消耗时间 4.64 mins
当前已完成第139个列的缺失值填补， 消耗时间 13.11 mins
当前已完成第89个列的缺失值填补， 消耗时间 13.59 mins
当前已完成第131个列的缺失值填补， 消耗时间 13.47 mins
当前已完成第18个列的缺失值填补， 消耗时间 13.52 mins
当前已完成第120个列的缺失值填补， 消耗时间 13.8 mins
当前已完成第123个列的缺失值填补， 消耗时间 12.89 mins
当前已完成第85个列的缺失值填补， 消耗时间 13.96 mins
当前已完成第7个列的缺失值填补， 消耗时间 13.64 mins
当前已完成第3个列的缺失值填补， 消耗时间 14.09 mins
当前已完成第6个列的缺失值填补， 消耗时间 14.22 mins
当前已完成第76个列的缺失值填补， 消耗时间 14.65 mins
当前已完成第32个列的缺失值填补， 消耗时间 7.75 mins
当前已完成第11个列的缺失值填补， 消耗时间 13.33 mins
当前已完成第138个列的缺失值填补， 消耗时间 14.08 mins
当前已完成第47个列的缺失值填补， 消耗时间 14.41 mins
当前已完成第75个列的缺失值填补， 消耗时间 13.75 mins
当前已完成第122个列的缺失值填补， 消耗时间 13.73 mins
当前已完成第121个列的缺失值填补， 消耗时间 13.84 mins
当前已完成第119个列的缺失值填补， 消耗时间 13.94 mins
当前已完成第20个列的缺失值填补， 消耗时间 8.07 mins
当前已完成第25个列的缺失值填补， 消耗时间 13.18 mins
当前已完成第124个列的缺失值填补， 消耗时间 13.15 mins
当前已完成第169个列的缺失值填补， 消耗时间 13.84 mins
当前已完成第159个列的缺失值填补， 消耗时间 14.27 mins
当前已完成第0个列的缺失值填补， 消耗时间 13.21 mins
当前已完成第39个列的缺失值填补， 消耗时间 14.11 mins
当前已完成第126个列的缺失值填补， 消耗时间 7.09 mins
当前已完成第167个列的

In [ ]:
train_df_randomForest[train_df_randomForest.isna().any(axis=1)]

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145


In [ ]:
train_df_randomForest = pd.concat([tmp, train_df_randomForest], axis = 1)

Convert DataFrame into parquet format

In [ ]:
train_df_randomForest.to_parquet(path = '../data/4-PreCompressed/FilledWithRandomForest/train_RandomForest(PreCompressed).parquet', engine = 'pyarrow')

In [ ]:
train_df_randomForest.to_parquet(path = 'train_RandomForest(PreCompressed).parquet', engine = 'pyarrow')